In [1]:
import datetime
print("Current Date/Time: ", datetime.datetime.now())

Current Date/Time:  2020-09-26 15:33:35.018521


In [2]:
!git clone https://github.com/millermuttu/TSAI-EVA5.git
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

Cloning into 'TSAI-EVA5'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 174 (delta 64), reused 100 (delta 26), pack-reused 0
Receiving objects: 100% (174/174), 828.54 KiB | 1.11 MiB/s, done.
Resolving deltas: 100% (64/64), done.
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 952kB 8.5MB/s 
albumentations==0.4.6 is successfully installed


In [3]:
cd /content/TSAI-EVA5/week9/

/content/TSAI-EVA5/week9


In [4]:
import torch
import dataset
import models
import utils
import numpy as np
from torchvision import transforms
from train_test import train, test

In [5]:
cuda, device = utils.others.initialize_device(utils.config.SEED)

GPU Available? True


In [6]:
trainset = dataset.pytorch_datasets(True, True, dataset_name='cifar10')
testset = dataset.pytorch_datasets(False, True, dataset_name='cifar10')

Extracting /content/TSAI-EVA5/week9/dataset/cifar10/cifar-10-python.tar.gz to /content/TSAI-EVA5/week9/dataset/cifar10
Files already downloaded and verified


In [7]:
mean, std = utils.img_reg.get_stats(trainset)[0], utils.img_reg.get_stats(trainset)[1]
print(mean, std)

(0.4914, 0.4822, 0.4465) (0.247, 0.2435, 0.2616)


In [8]:
transformations_list = [transforms.Normalize(mean, std)]
augmentations_list = [transforms.RandomRotation(7), 
                      transforms.RandomHorizontalFlip(0.3)]

In [9]:
trainset = dataset.pytorch_datasets(True, True, transformations_list, augmentations_list, dataset_name='cifar10')
testset = dataset.pytorch_datasets(False, True, transformations_list, dataset_name='cifar10')
train_dl = dataset.data_loader(trainset, utils.config.BATCH_SIZE, utils.config.NUM_WORKERS, cuda, shuffle=True)
test_dl = dataset.data_loader(testset, utils.config.BATCH_SIZE, utils.config.NUM_WORKERS, cuda, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [12]:
model = models.Quiz_net().to(device)

In [13]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Quiz_net().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           9,216
              ReLU-6           [-1, 32, 32, 32]               0
       BatchNorm2d-7           [-1, 32, 32, 32]              64
           Dropout-8           [-1, 32, 32, 32]               0
         MaxPool2d-9           [-1, 32, 16, 16]               0
           Conv2d-10           [-1, 64, 16, 16]          18,432
             ReLU-11           [-1, 64, 16, 16]               0
      BatchNorm2d-12           [-1, 64, 16, 16]             128
          Dropout-13           [-1, 64, 16, 16]               0
           Conv2d-14           [-1

In [14]:
model = Quiz_net().to(device)
loss_fn = utils.others.cross_entropy_loss_fn()
optimizer = utils.others.sgd_optimizer(model)
scheduler = utils.others.StepLR_scheduler(optimizer, step_size=6)
if utils.config.DEBUG == True:
    utils.config.EPOCHS = 15

test_loss = []
test_accuracy = []
train_losses = []
train_accuracy = []
misclassified_imgs = []
for epoch in range(utils.config.EPOCHS):
    print(f"Running Epoch {epoch+1}/{utils.config.EPOCHS}\n")
    train(model, train_dl, optimizer, loss_fn, device, train_losses, train_accuracy)
    scheduler.step()
    test(model, test_dl, loss_fn, device, 25, test_loss, test_accuracy, misclassified_imgs)

  0%|          | 0/1563 [00:00<?, ?it/s]

Running Epoch 1/15



  0%|          | 0/1563 [00:00<?, ?it/s]


Validation set: Average loss: 0.0321, Accuracy: 6362/10000 (63.62%)

Running Epoch 2/15



  0%|          | 0/1563 [00:00<?, ?it/s]

...


Validation set: Average loss: 0.0124, Accuracy: 8651/10000 (86.51%)

Running Epoch 14/15



  0%|          | 0/1563 [00:00<?, ?it/s]


Validation set: Average loss: 0.0125, Accuracy: 8629/10000 (86.29%)

Running Epoch 15/15



Loss=0.28 Batch_ID=1562 Accuracy=87.75: 100%|██████████| 1563/1563 [00:46<00:00, 33.54it/s]



Validation set: Average loss: 0.0125, Accuracy: 8642/10000 (86.42%)

